In [9]:
## Instalaciones

%pip install torch
%pip install open3d

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Using cuda.


In [ ]:
classes = [ModelNetClass.TOILET, ModelNetClass.MONITOR]     # antonio hice un error y sí o sí el indice en este arreglo
                                                            # determina el valor de 'y' en el mapeo de las clases
train_data = ModelNet(classes, True, False)

# parametros del dataset
batch_size = 16
dim = 3
num_points = 1024
num_classes = 2

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas
epochs = 200
learning_rate = 0.01
reg_weight = 0.001


train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
criterion = PointNetLoss(alpha=None, gamma=1, reg_weight=reg_weight, size_average=True).to(DEVICE)

loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "val_loss": {
        "loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    # Modo entrenamiento
    classifier = classifier.train()

    batch_losses = list()
    batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A)
                
        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()

        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas
        correct = pred_choice.eq(labels.data).cpu().sum()
        accuracy = correct.item() / float(pcds.size(0))

        # Registramos métricas para el batch
        batch_losses.append(loss.item() * pcds.size(0))
        batch_accs.append(accuracy)
        
    epoch_train_loss = np.mean(batch_losses)
    epoch_train_acc = np.mean(batch_accs)
    
    print("Loss", epoch_train_loss)
    print("Acc", epoch_train_acc)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_loss)


NameError: name 'true' is not defined

In [ ]:


# parametros del dataset
batch_size = 32             # número de point clouds
dim = 3                     # número de dimensiones por cada punto
num_points = 1024           # número de puntos por point cloud
num_classes = 2             # número de clases de clasificación

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas

test_data = torch.rand(batch_size, dim, num_points)
print(test_data.shape)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes)
out, _, _ = classifier(test_data)
print(f'Class output shape: {out.shape}')
print(f'Class output: {out}')

train_test(50, 0.01, 0.001, num_classes)